<a href="https://colab.research.google.com/github/aidanjmaldonado/penny-stock-lstm/blob/main/penny_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [678]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
import sqlite3
import requests
import sys
import pytz

from library.DataSetProcessor import DataSetProcessor
import matplotlib.pyplot as plt

import pickle







# Create database

In [679]:
# Download database from github repository
# historical_url = "https://raw.githubusercontent.com/CSE-115-UCSC/penny-stock-lstm/main/historicaldata.db"
# scrape_request = requests.get(historical_url)

try:
  # Contingent on request status
  # scrape_request = requests.get(historical_url)
  # scrape_request.raise_for_status()

  # Create local database from pull, name 'historicaldata.db'
  with open("historical.db", "wb") as db_file:
    db_file.write(scrape_request.content)

  print("Request to download database succeeded")


except:
  # Report failed request status
  sys.stderr.write("Request to download database failed")

Request to download database succeeded


In [680]:
dataSetProcessor = DataSetProcessor()

In [681]:
# Connect to SQlite database
try:
    db = 'historical.db'
    sqliteConnection = sqlite3.connect(db)
    cursor = sqliteConnection.cursor()
    print(f'SQlite connected with {db}')

except:
    sys.stderr.write("Failed to connect to database")

SQlite connected with historical.db


In [682]:
# Query {ticker(s)} from Database
try:
    query = f"SELECT * all_historical;"
    cursor.execute(query)
    if cursor.fetchone() is None:
        raise Exception("No results")

    print(f"Success querying all historical")
    # Turn SQlite Database into Pandas Dataframe
    data = pd.read_sql_query(query, sqliteConnection)

except:
    sys.stderr.write(f"Failed to select all historical")

Failed to select all historical

In [683]:
# # Using the column 'time' (millisecond) add a new column 'dates' with datetime
dates = pd.to_datetime(data['time'], unit='ms')
tickers = data['ticker']
dates = dates.dt.tz_localize('UTC').dt.tz_convert('US/Pacific')
dates = dates.dt.tz_localize(None)
data = dataSetProcessor.addDateAndMinutes(data)

In [684]:
data.head()

,id,ticker,volume,volume_weighted_average,open,close,high,low,time,number_of_trades,date,timeTime,timeMinute
0,1,ACHR,269.0,3.0307,3.0301,3.030,3.0301,3.0300,1657643400000,4,2022-07-12,09:30:00,930
1,2,ACHR,2037.0,3.0362,3.0400,3.040,3.0400,3.0350,1657643700000,35,2022-07-12,09:35:00,935
2,3,ACHR,796.0,3.0364,3.0400,3.035,3.0400,3.0350,1657644000000,17,2022-07-12,09:40:00,940
3,4,ACHR,1795.0,3.0356,3.0350,3.035,3.0400,3.0350,1657644300000,19,2022-07-12,09:45:00,945
4,5,ACHR,10397.0,3.0286,3.0350,3.025,3.0350,3.0212,1657644600000,87,2022-07-12,09:50:00,950


In [685]:
# # Scale *For every ticker separately to (0, 1)
# data['closeNorm'] = data['close'] /  data['close'].max()

In [686]:
# # Begin plotting
# import matplotlib.pyplot as plt
# plt.figure()

# # 10 different tickers
# for ticker in tickers.unique():
#     plt.plot(dates[tickers == ticker], data[data['ticker'] == ticker]['closeNorm'], label=ticker)
#     plt.label = ticker
#     plt.legend()
#     plt.show()

# Dataset Normalization

In [687]:
# achrData = data[data['ticker'] == 'ACHR']
# len(achrData)
# plt.plot(achrData['close'] / achrData['close'].max())

In [688]:
normalized_data = pd.DataFrame(columns=['close', 'time', 'volume'])

In [689]:
dataByTicker = {}
for ticker in data['ticker'].unique():
    dataByTicker[ticker] = data[data['ticker'] == ticker].copy()
    dataByTicker[ticker]['closeNorm'] = dataByTicker[ticker]['close'] / dataByTicker[ticker]['close'].max()
    dataByTicker[ticker]['volumeNorm'] = dataByTicker[ticker]['volume'] / dataByTicker[ticker]['volume'].max()

In [690]:
for key in dataByTicker:
    # Create a temporary DataFrame to hold the current data
    temp_df = pd.DataFrame({
        'close': dataByTicker[key]['closeNorm'],
        # 'ticker': dataByTicker[key]['ticker'],
        'volume': dataByTicker[key]['volumeNorm'],
        'time': dataByTicker[key]['time']
    })
    
    # Concatenate the temporary DataFrame to the normalized_data DataFrame
    normalized_data = pd.concat([normalized_data, temp_df], ignore_index=True)

# Optionally, you can reset the index if needed
normalized_data.reset_index(drop=True, inplace=True)

/var/folders/s4/t5945xms4x52fkpsd0m4v9340000gn/T/ipykernel_77387/273319749.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  normalized_data = pd.concat([normalized_data, temp_df], ignore_index=True)


In [691]:
normalized_data.head()

,close,time,volume
0,0.383059,1657643400000,0.000090
1,0.384324,1657643700000,0.000678
2,0.383692,1657644000000,0.000265
3,0.383692,1657644300000,0.000598
4,0.382427,1657644600000,0.003461


In [692]:
# for ticker in data['ticker'].unique():
#     tickerD = dataByTicker[ticker]
#     plt.plot(tickerD['closeNorm'])
#     plt.plot(tickerD['volumeNorm'])
#     plt.title(ticker)
#     plt.show()

# Train on all historical stock data, sequenced

In [693]:
# Select features to use
# features = data[['volume', 'volume_weighted_average', 'open', 'close', 'high', 'low']].values

# Normalize features
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_features = scaler.fit_transform(features)

def create_sequences(data, sequence_length, prediction_length):
    xs, ys = [], []
    index = 0
    count = 0
    while index < len(data) - sequence_length - prediction_length + 1:
        # Check if sequence is within a single day
        if dates[index].date() == dates[index + sequence_length].date() and tickers[index] == tickers[index + sequence_length]:
            # If day == 2024-05-31, print
            # if dates[index] == "2024-05-31":
            # print("We got a sequence from", dates[index], "to", dates[index + sequence_length], "sequence-length is", (index + sequence_length) -index, tickers[index], tickers[index + sequence_length])
            xs.append(data.iloc[index:index + sequence_length])  # Use past data for features
            ys.append(data.iloc[index + sequence_length:index + sequence_length + prediction_length, 0])  # Only predict 'close' prices
            index += sequence_length
            count += 1
        else:  # Move index to the start of the next day
            newindex = index
            while dates[newindex].date() == dates[newindex + 1].date():
                newindex += 1
            newindex += 1
            index = newindex
    print("Valid days:", count)
    return np.array(xs), np.array(ys)

# One day sequence lenght
sequence_length = 78
prediction_length = 78
x, y = create_sequences(normalized_data, sequence_length, prediction_length)

train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Split data into train and test sets
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

# Type adjustment
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)

# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(sequence_length, x_train.shape[2])))
model.add(LSTM(50))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(prediction_length))

model.compile(optimizer='adam', loss='mse')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model
# model.evaluate(x_test, y_test)

Valid days: 294
x_train shape: (235, 78, 3)
y_train shape: (235, 78)
x_test shape: (59, 78, 3)
y_test shape: (59, 78)
Epoch 1/100


/Users/aidanmaldonado/Documents/PennyStock/Website/stock-prediction/.venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.2392 - val_loss: 0.0414
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1755 - val_loss: 0.0353
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1717 - val_loss: 0.0318
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1469 - val_loss: 0.0273
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1365 - val_loss: 0.0285
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1319 - val_loss: 0.0253
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1107 - val_loss: 0.0270
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0918 - val_loss: 0.0306
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0829 - val_loss: 0.0291
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0870 - val_loss: 0.0396
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0828 - val_loss: 0.0252
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0801 - val_loss: 0.0327
E

In [694]:
model.save('model.h5')

# Prediction on unseen 'Real-Time' stock


In [695]:
# Create Prediction Stock dataset

try:
  query = f"SELECT * FROM all_historical WHERE ticker = 'SOUN';" # *NOTE* Cannot run predictions on EVGN_Predict rn because it has 0 valid days, using GSAT
  cursor.execute(query)
  if cursor.fetchone() is None:
    raise Exception("No results")
  print(f"Success querying EVGN_Predict")
except:
  sys.stderr.write(f"Failed to select EVGN_Predict")

# Turn SQlite Database into Pandas Dataframe
predict_data_whole = pd.read_sql_query(query, sqliteConnection)

# Split into known vs unknown, where unknown is the last 78 entries
split_index_whole = len(predict_data_whole) - 78
known_data_whole = predict_data_whole.iloc[:-78]
unknown_data_whole = predict_data_whole.iloc[-78:]

# # Features
# known_features = known_data_whole[['volume', 'volume_weighted_average', 'open', 'close', 'high', 'low']].values
known_features = known_data_whole[['close', 'volume', 'time']].values
known_closes = np.array([known_features[:, 0]]).T
known_volume = np.array([known_features[:, 1]]).T


 # Normalize features
predict_close_scaler = MinMaxScaler(feature_range=(0, 1))
predict_volume_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close = predict_close_scaler.fit_transform(known_closes)
scaled_volume = predict_volume_scaler.fit_transform(known_volume)

scaled_known_features = known_features
known_features[:, 0] = scaled_close[:, 0]
known_features[:, 1] = scaled_volume[:, 0]



Success querying EVGN_Predict


In [696]:
# Create dates column from miliseconds
dates = pd.to_datetime(known_data_whole['time'], unit='ms')
tickers = known_data_whole['ticker']
dates = dates.dt.tz_localize('UTC').dt.tz_convert('US/Pacific')
dates = dates.dt.tz_localize(None)

In [697]:
# Create Prediction Sequences
sequence_length = 78
prediction_length = 78

def create_predict_sequences(data, sequence_length):
    index = len(data) - sequence_length
    xs = []
    xs.append(data[index:])  # Use the last sequence_length data points
    return np.array(xs)
x_predict = create_predict_sequences(scaled_known_features, sequence_length)
x_predict.shape


(1, 78, 3)

In [698]:
# Predict with model

predicted_stock_sequence = model.predict(x_predict)
print(predicted_stock_sequence)

# Unscale
final_predictions = predict_close_scaler.inverse_transform(predicted_stock_sequence)

# Evaluate the model
# model.evaluate(x_predict, final_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
[[-0.05620649  0.41039336  0.17958891  0.2635536   0.2670682  -0.01043692
   0.20391582  0.08646833  0.21371108  0.35968563  0.17359357  0.27523324
   0.18025392  0.16102016 -0.01483409  0.10473824  0.090024    0.42727837
   0.5407835   0.36467436 -0.02439497 -0.15642795  0.07467686  0.27039087
   0.19405851  0.10414981  0.03392808 -0.08552811  0.1426267   0.01033249
   0.4243185  -0.00358766  0.27506712  0.28662845 -0.07683682 -0.00881347
   0.00889271  0.28080887  0.08607534  0.53982115  0.17005943  0.36823606
   0.03673975  0.09578745 -0.05523349  0.01434146  0.17640726  0.29126042
  -0.4000519   0.12352055  0.33578438  0.14722902  0.32443678  0.2262901
  -0.03622572  0.14515404  0.13219945  0.48129463  0.12100008  0.08915223
   0.27175206  0.0807673   0.0898499   0.38310003  0.12331612  0.01984584
   0.35692438  0.13838616  0.12006585  0.2918186   0.13872707  0.3376459
  -0.03924361  0.23867214  0.06930283 -0.17456146  0.2827219   0.07030544]]

In [699]:
# Plot results
import matplotlib.pyplot as plt

# Create the combined index for plotting
combined_index = np.arange(len(predict_data_whole))

# Create
plt.figure(figsize=(14, 7))

# Plot
plt.plot(combined_index, predict_data_whole['close'], label='Example "Prediction" Data', color='black', linewidth=3.0)
plt.plot(combined_index[:split_index_whole], known_data_whole['close'], label='"Known" Data', color='cyan', linewidth=2.0, linestyle="--")
plt.plot(combined_index[split_index_whole:], unknown_data_whole['close'], label='"Unknown" Data', color='orange', linewidth=2.0, linestyle="--")

plt.plot(combined_index[split_index_whole:], final_predictions[0], label='Model Prediction', color='purple', linewidth=2.0, linestyle="-")



# Format
plt.title('Comparison of Historical, Actual, and Predicted Data')
plt.xlabel('Time Steps')
plt.ylabel('Stock Price')
plt.legend()

# Adjust y-axis limits based on the data range
# plt.ylim(predict_data_whole.iloc[-200:]['close'].min() - 0.2, predict_data_whole.iloc[-200:]['close'].max() + 0.2)
plt.xlim(len(predict_data_whole) - 200, len(predict_data_whole))  # Crop view to just the very end

# Set y-tick locations with a step of 0.05
# yticks = np.arange(min(predict_data_whole['close']) // 0.05 * 0.05, (max(predict_data_whole['close']) // 0.05 + 1) * 0.05, 0.05)
# plt.yticks(yticks)

plt.show()
